In [1]:
import os
import sys
import copy
import wget
import time
import asyncio
import warnings
import logging

import numpy as np

from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel.attcs import ATTCS

%matplotlib inline

In [2]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [3]:
attcs = ATTCS()
# cam = salobj.Remote(attcs.domain, "GenericCamera", 1)

await attcs.start_task

Read historical data in 0.05 sec
Read historical data in 1.41 sec
Read historical data in 2.35 sec
Read historical data in 3.49 sec
Read historical data in 4.29 sec
Read historical data in 5.35 sec
Read historical data in 6.47 sec
RemoteTelemetry(ATDome, 0, position) falling behind; read 29 messages


[None, None, None, None, None, None, None]

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 15 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 17 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 18 messages


In [4]:
# Declare target name
target_name='HD32309'

RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 19 messages


In [8]:
await attcs.enable(settings={
    'ataos': "current",
    'atmcs': "",
    'atptg': "",
    'atpneumatics': "",
    'athexapod': "auxtel_201912_crazy.yaml",
    'atdome': "test.yaml",
    'atdometrajectory': ""})

Gathering settings.
Received settings from users.: {'ataos': 'current', 'atmcs': '', 'atptg': '', 'atpneumatics': '', 'athexapod': 'auxtel_201912_crazy.yaml', 'atdome': 'test.yaml', 'atdometrajectory': ''}
Settings versions: {'ataos': 'current', 'atmcs': '', 'atptg': '', 'atpneumatics': '', 'athexapod': 'auxtel_201912_crazy.yaml', 'atdome': 'test.yaml', 'atdometrajectory': ''}
Enabling all components
Enabling  atmcs
Enabling  atptg
Enabling  ataos
Enabling  atpneumatics
Enabling  athexapod
Enabling  atdome
Enabling  atdometrajectory
[atmcs]::[<State.ENABLED: 2>]
[atptg]::[<State.ENABLED: 2>]
[ataos]::[<State.ENABLED: 2>]
[atpneumatics]::[<State.ENABLED: 2>]
[athexapod]::[<State.ENABLED: 2>]
[atdome]::[<State.ENABLED: 2>]
[atdometrajectory]::[<State.ENABLED: 2>]
All components enabled.


In [ ]:
rotation_matrix = lambda angle: np.array([
            [np.cos(np.radians(angle)), -np.sin(np.radians(angle)), 0.],
            [np.sin(np.radians(angle)), np.cos(np.radians(angle)), 0.],
            [0., 0., 1.]])

In [9]:
await attcs.slew_icrs(ra = "5 01 25.58", dec = "-20 03 06.9", rot_pa=-155.-88-48.-8-7, target_name=target_name)

Flushing events
Sending command
Scheduling check coroutines
process as completed...
[Telescope] delta Alt = -000.332 | delta Az= -000.474 [Dome] delta Az = -000.263
[Telescope] delta Alt = -000.043 | delta Az= -000.106 [Dome] delta Az = -000.263
[Telescope] delta Alt = +000.001 | delta Az= +000.001 [Dome] delta Az = -000.263


AttributeError: 'NoneType' object has no attribute 'private_sndStamp'

Axes in position.


In [ ]:
-1./4200.*-125, -1./131*-60

In [ ]:
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(x=0.,y=0.,z=0.02)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

In [ ]:
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(x=-0.5,y=0.4,z=0.)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

In [10]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=49.-41.22144+27, el=16.+35-21.6, num=0) 

In [15]:
-1./4200.*50.

-0.011904761904761904

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages


In [ ]:
x = -60
y = +80

In [14]:
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(x=0.,y=0.,z=-0.03)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

In [ ]:
-54.*0.4

In [ ]:
-54*0.648854961832061

In [ ]:
await attcs.ataos.cmd_offset.set_start(x=0.,y=0.,z=-0.8)



If rotator is aligned with boresight:

 +El-offset = -x-axis

 +Az-offset = -y-axis
 
 hexapod movement in y is movement in x
 

In [ ]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=49., el=16., num=0)

In [ ]:
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20, -18, 0.], rotation_matrix(angle))
print(az,el)
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)


In [ ]:
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20-90., -18, 0.], rotation_matrix(angle))
print(az,el)
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)



In [ ]:
offset_x = (np.random.random()-0.5)*240.
offset_y = (np.random.random()-0.5)*240.
print(offset_x, offset_y)
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+offset_x, -18+offset_y, 0.], rotation_matrix(angle))
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)


In [ ]:
c = salobj.Controller("Script", index=11)

In [ ]:
await c.start_task

In [ ]:
offset_x = (np.random.random()-0.5)*240.
offset_y = (np.random.random()-0.5)*240.
print(offset_x, offset_y)
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+offset_x, -18+offset_y, 0.], rotation_matrix(angle))
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)



In [ ]:
offset_x = (np.random.random()-0.5)*240.
offset_y = (np.random.random()-0.5)*240.
print(offset_x, offset_y)
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+offset_x, -18+offset_y, 0.], rotation_matrix(angle))
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [ ]:
offset_x = (np.random.random()-0.5)*240.
offset_y = (np.random.random()-0.5)*240.
print(offset_x, offset_y)
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+offset_x, -18+offset_y, 0.], rotation_matrix(angle))
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [ ]:
offset_x = (np.random.random()-0.5)*240.
offset_y = (np.random.random()-0.5)*240.
print(offset_x, offset_y)
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+offset_x, -18+offset_y, 0.], rotation_matrix(angle))
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [ ]:
offset_x = (np.random.random()-0.5)*240.
offset_y = (np.random.random()-0.5)*240.
print(offset_x, offset_y)
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+offset_x, -18+offset_y, 0.], rotation_matrix(angle))
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [ ]:
await attcs.ataos.cmd_offset.set_start(z=+0.05)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

In [ ]:
await attcs.enable(settings={
    'ataos': "current",
    'atmcs': "",
    'atptg': "",
    'atpneumatics': "",
    'athexapod': "auxtel_201912_crazy.yaml",
    'atdome': "test.yaml",
    'atdometrajectory': ""})

In [ ]:
rotation_matrix = lambda angle: np.array([
            [np.cos(np.radians(angle)), -np.sin(np.radians(angle)), 0.],
            [np.sin(np.radians(angle)), np.cos(np.radians(angle)), 0.],
            [0., 0., 1.]])
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20, -18., 0.], rotation_matrix(angle))

In [ ]:
el, az

In [ ]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [ ]:
5*360/24

In [ ]:
attcs.slew_icrs?

In [ ]:
await attcs.slew_icrs

In [ ]:
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="x", offset=x_offset)
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="y", offset=y_offset)

In [ ]:
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=focus_offset)

In [ ]:
dz = 0.8

try:
    await cam.cmd_stopLiveView.start()
except salobj.AckError:
    print("ack error")

# Intrafocal hexapod setting - offsets -0.5mm from central focus value above

print('Move to intrafocal position')
# await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
#                                              z=hex_Z-dz, u=hex_U, v=hex_V)
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=-dz+focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

print('Take intrafocal image')
cam.evt_endReadout.flush()

await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_intra')

intra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

# Extrafocal hexapod setting
print('Move to extra-focal hexapod position')
# await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
#                                                      z=hex_Z+dz, u=hex_U, v=hex_V)
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=dz+focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

# r.evt_endReadout.flush()
print('Take extra-focal image')
cam.evt_endReadout.flush()
await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_extra')

extra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()

print(f"============")
print(f"dz={dz}")
print(intra_endron.imageName)
print(extra_endron.imageName)
print(f"============")

try:
    await cam.cmd_startLiveView.set_start(expTime=0.5)
except salobj.AckError:
    print("ack error")

attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

print(f"['{intra_endron.imageName}.fits'], ['{extra_endron.imageName}.fits']")
print(np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle))

# intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
# extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'
# intra_fname = wget.download(intra_wget_url)
# print(f"============")
# print(f'{intra_fname}')
# extra_fname = wget.download(extra_wget_url)
# print(f'{extra_fname}')
# print(f"============")


In [ ]:
intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'

intra_fname = wget.download(intra_wget_url, out="/home/saluser/develop/cwfs/python/data")
extra_fname = wget.download(extra_wget_url, out="/home/saluser/develop/cwfs/python/data")

print(intra_fname,extra_fname)

In [ ]:
await attcs.ataos.cmd_offset.set_start(z=0.)

In [ ]:
await attcs.ataos.cmd_resetOffset.start()

In [ ]:
await attcs.ataos.cmd_offset.set_start(z=-0.1)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)
print("done")

In [ ]:
await attcs.atdome.cmd_openShutter.start()

In [1]:
print()